In [1]:
import random
# from hdflogv2 import HDFSLogv2
from hdflogv3 import HDFSLogv3
import tensorflow as tf
import numpy as np
import pandas as pd
from keras.preprocessing.sequence import pad_sequences
import pickle

In [2]:
creating an instance of the hdfslogs, we will have to inspect the sequence and the loglines later to 
tune the values of sequence length and the characters in line. 
hlog_meta_176_32 = HDFSLogv3(train_ratio=0.8, padded_char_len=176, padded_seq_len=32,
                rm_time_stamp=False, rm_ip_address=False, debug=True )

In [3]:
get_train_test_data_text is to see the texts in sequence , for model training get_train_test_data_num will be used
# the ablation parameter allows to take only 10 number each from positive and negative sequences
x_train, y_train, x_test, y_test = hlogs.get_padded_train_test_data(ablation=1000, save_pkl=True,
                                                                   hdfs_pkl_name='hdfsv3_32_176.pkl')

In [4]:
hlog_no_meta_176_32 = HDFSLogv3(train_ratio=0.8, padded_char_len=176, padded_seq_len=32,
                rm_time_stamp=True, rm_ip_address=True, debug=True )
x_train, y_train, x_test, y_test = hlogs.get_padded_train_test_data(ablation=1000, save_pkl=True,
                                                                   hdfs_pkl_name='hdfsv3_32_176_no_meta.pkl')

In [5]:
# x_data = hlogs.labelled_num_seq_df['LogNumSequence'].values
# y_data = hlogs.labelled_num_seq_df['Label'].values
# pos_idx = y_data > 0
# x_pos = x_data[pos_idx]
# y_pos = y_data[pos_idx]
# x_neg = x_data[~pos_idx]
# y_neg = y_data[~pos_idx]
# ablation = 1000
# train_ratio = 0.8


# if ablation != 0 and x_pos.shape[0] >= ablation:
#     x_pos = x_pos[0:ablation]
#     y_pos = y_pos[0:ablation]
#     x_neg = x_neg[0:ablation]
#     y_neg = y_neg[0:ablation]
#     print('ablation x_pos:', x_pos.shape)


In [6]:
class HDFS:
    
    def __init__(self, padded_num_seq_df, ablation=100, train_ratio=0.8, 
                 val_ratio=None, test_ratio=None, designated_ukc_cls=None, 
                 debug=True):
        self.padded_num_seq_df = padded_num_seq_df
        self.ablation = ablation
        self.train_ratio = train_ratio
        self.val_ratio = val_ratio
        self.test_ratio = test_ratio
        self.designated_ukc_cls = designated_ukc_cls
        self.debug = debug        
    
    
    def get_train_test_split_single_class(self, label=0):
        bgldf = self.padded_num_seq_df
        bgldf.rename(columns = {'Label':'label', 'LogNumSequence': 'seq'}, inplace = True)
        bgldf.drop(['BlockId'], axis = 1, inplace = True)
        train_data = None
        val_data = None
        test_data = None
        ukc_data = None ### unknown known - not present in the training data but present in the test data        
        
        train_cnt = round(self.ablation * self.train_ratio)#### 1000 * 0.7 = 700
        remaining_cnt = round(self.ablation *(1 - self.train_ratio)) ### 1000 * (1-0.7) = 300 
        if self.val_ratio is None and self.test_ratio is None:
            val_cnt = test_cnt = remaining_cnt//2 ### 300/2 = 150 each
        else:
            val_cnt = round(self.ablation * self.val_ratio) ### 1000 * 0.2 = 200 
            test_cnt = round(self.ablation * self.test_ratio) ### 1000 * 0.1 = 100
        
        cls_data = bgldf[bgldf.label==label]
        cls_data_cnt = cls_data.count()[0]
        cls_unique_label = int(np.unique(cls_data.label)[0])
        #if self.debug: print('cls_unique_label', cls_unique_label)
        if self.designated_ukc_cls == cls_unique_label:
            if cls_data_cnt < test_cnt:
                ukc_data = cls_data[0:cls_data_cnt]
            else:
                ukc_data = cls_data[0:test_cnt]
            print(f'class {cls_unique_label} is added as ukc')
        else:
            if self.ablation <= cls_data_cnt: ### if 1000 <= 2000            
                train_data = cls_data[0:train_cnt] ### cls_data[0:700]
                val_data = cls_data[train_cnt:train_cnt+val_cnt] ### cls_data[700:(700+200)]
                test_data = cls_data[train_cnt+val_cnt:self.ablation] ### cls_data[900:1000]
            elif self.ablation > cls_data_cnt and cls_data_cnt >= train_cnt+val_cnt: ### 1000>950 and 950>(700+200)
                train_data = cls_data[0:train_cnt] ### cls_data[0:700]
                remaining_for_test = cls_data_cnt - (train_cnt+val_cnt) ### 950 - (700+200) = 50
                if remaining_for_test > 0: ### 50 > 0
                    val_data = cls_data[train_cnt:train_cnt+val_cnt] ### cls_data[700:(700+200)]
                    test_data = cls_data[train_cnt+val_cnt:cls_data_cnt] ### cls_data[900:950]
                else: ### cls_data_cnt = 850 or 900
                    val_data = cls_data[train_cnt:cls_data_cnt] ### cls_data[700:850]
            else:
                if self.debug:
                    print(f'{cls_data_cnt} data in class {label} not enough to split into train:{train_cnt} and validation:{val_cnt}, adding the entire data as ukc')
                if self.designated_ukc_cls is None:    
                    if cls_data_cnt < test_cnt:
                        ukc_data = cls_data[0:cls_data_cnt]
                    else:
                        ukc_data = cls_data[0:test_cnt]
        # if self.debug:    
        if train_data is not None:
            print(f'train_{label}:, {train_data.count()[0]}', end=', ')
        if val_data is not None:
            print(f'val_{label}:, {val_data.count()[0]}', end=', ')
        if test_data is not None:
            print(f'test_{label}:, {test_data.count()[0]}', end=', ')
        if ukc_data is not None:
            print(f'ukc_{label}:, {ukc_data.count()[0]}')
        return train_data, val_data,test_data, ukc_data
    

In [7]:
# 
# with open('hlog.pkl', 'wb') as f:
#     pickle.dump(hlogs, f)

In [8]:
file_name = os.path.join('data', 'hlog_no_meta_176_32.pkl')
with open(file_name, 'rb') as f:
    hlog_no_meta_176_32 = pickle.load(f)

In [9]:
# hlogs.labelled_num_seq_df

In [10]:
hdfs = HDFS(hlog_no_meta_176_32.labelled_num_seq_df)

In [11]:
hdfs.get_train_test_split_single_class(label=0)

train_0:, 80, val_0:, 10, test_0:, 10, 

(                                                  seq  label
 0   [[3, 14, 2, 2, 2, 2, 3, 28, 20, 20, 11, 6, 6, ...      0
 1   [[3, 14, 2, 2, 2, 3, 3, 6, 3, 2, 3, 22, 21, 2,...      0
 2   [[3, 14, 2, 2, 2, 2, 3, 28, 20, 27, 2, 11, 6, ...      0
 3   [[3, 14, 2, 2, 2, 3, 3, 2, 11, 2, 2, 27, 11, 1...      0
 4   [[3, 14, 2, 2, 2, 3, 6, 6, 2, 20, 21, 2, 2, 11...      0
 ..                                                ...    ...
 77  [[3, 14, 2, 2, 2, 3, 3, 3, 2, 2, 6, 22, 6, 22,...      0
 78  [[3, 14, 2, 2, 2, 2, 3, 28, 21, 14, 21, 2, 6, ...      0
 79  [[3, 14, 2, 2, 2, 2, 3, 27, 6, 14, 3, 2, 6, 21...      0
 80  [[3, 14, 2, 2, 2, 2, 3, 22, 11, 22, 6, 22, 6, ...      0
 81  [[3, 14, 2, 2, 2, 3, 3, 3, 21, 11, 21, 28, 6, ...      0
 
 [80 rows x 2 columns],
                                                   seq  label
 82  [[3, 14, 2, 2, 2, 2, 2, 3, 11, 27, 20, 28, 6, ...      0
 83  [[3, 14, 2, 2, 2, 2, 3, 28, 2, 11, 3, 6, 6, 11...      0
 84  [[3, 14, 2, 2, 3, 28, 6, 21, 11, 2, 3, 

In [16]:
print('hdfs log object pks file size in GB: ', 23722371/(1024*1024))

hdfs log object pks file size in GB:  22.62341594696045
